In [ ]:
# import the necessary packages
import tensorflow as tf
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import precision_score, f1_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import PIL.Image

In [ ]:
INIT_LR = 1e-4
EPOCHS = 10
BS = 32

DIRECTORY = "dataset/"
CATEGORIES = ["with_mask", "without_mask"]

data = []
labels = []

print(f"parameters = [learning rate = {INIT_LR}, epochs = {EPOCHS}, batch size = {BS} ]")
print("The two classes are - with_mask, without_mask")

In [ ]:
print("loading images from dataset")
for category in CATEGORIES:
    path = os.path.join(DIRECTORY,category)
    label = CATEGORIES.index(category)
    # resize and add labels
    for file in os.listdir(path):
        img_path = os.path.join(path,file)
        img = cv2.imread(img_path)
        img = cv2.resize(img,(224,224))
        data.append(img)
        labels.append(label)

print("Images and labels loaded.")
print(data)
print(labels)


In [ ]:
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)
print(labels)

data = np.array(data, dtype="float32")
labels = np.array(labels)
print("lenght of data - ", len(data))
print("lenght of labels - ", len(data))
print(data)
print(labels)
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.20, stratify=labels, random_state=42)
print("Number of training images - ", len(trainX))
print("Number of testing images - ", len(testX))

In [ ]:
# construct the training image generator for data augmentation
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [ ]:
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))
baseModel.summary()

In [ ]:
baseModel = MobileNetV3Small(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))
baseModel.summary()


In [ ]:
baseModel = DenseNet121(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))
baseModel.summary()


In [ ]:
baseModel = VGG19(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))
baseModel.summary()


In [ ]:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

In [ ]:
model = Model(inputs=baseModel.input, outputs=headModel)
model.summary()

In [ ]:
for layer in baseModel.layers:
	layer.trainable = False

In [ ]:
# compile our model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR,weight_decay=INIT_LR/BS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

In [ ]:
# train the head of the network
print("[INFO] training head...")
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=10)

In [ ]:
# Plotting accuracy curve
plt.plot(H.history['accuracy'], label='Training Accuracy')
plt.plot(H.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Curve')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Plotting loss curve
plt.plot(H.history['loss'], label='Training Loss')
plt.plot(H.history['val_loss'], label='Validation Loss')
plt.title('Loss Curve')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
train_preds = model.predict(trainX)
train_preds_binary = (train_preds > 0.5).astype(int)

train_precision = precision_score(trainY, train_preds_binary,average='micro')
train_f1 = f1_score(trainY, train_preds_binary,average='micro')

print(f'Training Precision: {train_precision:.4f}')
print(f'Training F1 Score: {train_f1:.4f}')

test_preds = model.predict(testX)
test_preds_binary = (test_preds > 0.5).astype(int)

test_precision = precision_score(testY, test_preds_binary,average='micro')
test_f1 = f1_score(testY, test_preds_binary,average='micro')

print(f'Testing Precision: {test_precision:.4f}')
print(f'Testing F1 Score: {test_f1:.4f}')

In [ ]:
# Save the trained model
print("[INFO] saving mask detector model...")
model.save("FMD_DenseNet121_D1.h5")